[Spark-TTS Github](https://github.com/SparkAudio/Spark-TTS) <br>
[Spark-TTS HuggingFace](https://huggingface.co/SparkAudio/Spark-TTS-0.5B)

⚠️ Usage Disclaimer: (Copied From Spark-TTS GitHub)<br>
This project provides a zero-shot voice cloning TTS model intended for academic research, educational purposes, and legitimate applications, such as personalized speech synthesis, assistive technologies, and linguistic research.

Please note:

Do not use this model for unauthorized voice cloning, impersonation, fraud, scams, deepfakes, or any illegal activities.

Ensure compliance with local laws and regulations when using this model and uphold ethical standards.

The developers assume no liability for any misuse of this model.

We advocate for the responsible development and use of AI and encourage the community to uphold safety and ethical principles in AI research and applications. If you have any concerns regarding ethics or misuse, please contact us.

In [ ]:
#@title Install Spark-TTS
%cd /content/
!git clone https://github.com/SparkAudio/Spark-TTS.git
%cd Spark-TTS
!pip install einops==0.8.1 einx==0.3.0 numpy==2.2.3 omegaconf==2.3.0 packaging==24.2 safetensors==0.5.2 soundfile==0.12.1 soxr==0.5.0.post1 tqdm==4.66.5 transformers==4.46.2 gradio==5.18.0
# !pip install torch==2.5.1
# !pip install torchaudio==2.5.1

from IPython.display import clear_output
clear_output()
print("Restart Session Complete, Run Next Cell")
import time
time.sleep(5)
#automatically restart session

import os
os.kill(os.getpid(), 9)


Restart Session Complete, Run Next Cell


In [5]:
#@title Fix BUG
import os

old_path = "/content/Spark-TTS/cli"
new_path = "/content/Spark-TTS/spark_cli"

# Rename 'cli' to 'spark_cli' if it exists
# Python was confusing with local cli/ folder with a system-installed cli package.
if os.path.exists(old_path):
    os.rename(old_path, new_path)

# Ensure '__init__.py' exists in 'spark_cli'
# Ensure Package Recognition – Adding __init__.py marks spark_cli/ as a proper package.
init_file = os.path.join(new_path, "__init__.py")
if not os.path.exists(init_file):
    open(init_file, "w").close()

print(f"Renamed '{old_path}' to '{new_path}' and ensured '__init__.py' exists.")


Renamed '/content/Spark-TTS/cli' to '/content/Spark-TTS/spark_cli' and ensured '__init__.py' exists.


In [6]:
#@title Download Model
from huggingface_hub import snapshot_download
snapshot_download("SparkAudio/Spark-TTS-0.5B", local_dir="/content/Spark-TTS/pretrained_models/Spark-TTS-0.5B")
from IPython.display import clear_output
clear_output()
print(f"Model Save at: /content/Spark-TTS/pretrained_models/Spark-TTS-0.5B")

Model Save at: /content/Spark-TTS/pretrained_models/Spark-TTS-0.5B


For Speech to Text try: [whisper-large-v3-turbo](https://huggingface.co/spaces/hf-audio/whisper-large-v3-turbo)

In [ ]:
#@title Run Gradio App
# Copyright (c) 2025 SparkAudio
#               2025 Xinsheng Wang (w.xinshawn@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
%cd /content/Spark-TTS
import os
import torch
import soundfile as sf
import logging
import argparse
import gradio as gr
from datetime import datetime
from spark_cli.SparkTTS import SparkTTS

from sparktts.utils.token_parser import LEVELS_MAP_UI


def initialize_model(model_dir="pretrained_models/Spark-TTS-0.5B", device=0):
    """Load the model once at the beginning."""
    logging.info(f"Loading model from: {model_dir}")
    device = torch.device(f"cuda:{device}")
    model = SparkTTS(model_dir, device)
    return model


def run_tts(
    text,
    model,
    prompt_text=None,
    prompt_speech=None,
    gender=None,
    pitch=None,
    speed=None,
    save_dir="example/results",
):
    """Perform TTS inference and save the generated audio."""
    logging.info(f"Saving audio to: {save_dir}")

    if prompt_text is not None:
        prompt_text = None if len(prompt_text) <= 1 else prompt_text

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Generate unique filename using timestamp
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    save_path = os.path.join(save_dir, f"{timestamp}.wav")

    logging.info("Starting inference...")

    # Perform inference and save the output audio
    with torch.no_grad():
        wav = model.inference(
            text,
            prompt_speech,
            prompt_text,
            gender,
            pitch,
            speed,
        )

        sf.write(save_path, wav, samplerate=16000)

    logging.info(f"Audio saved at: {save_path}")

    return save_path


def build_ui(model_dir, device=0):

    # Initialize model
    model = initialize_model(model_dir, device=device)

    # Define callback function for voice cloning
    def voice_clone(text, prompt_text, prompt_wav_upload, prompt_wav_record):
        """
        Gradio callback to clone voice using text and optional prompt speech.
        - text: The input text to be synthesised.
        - prompt_text: Additional textual info for the prompt (optional).
        - prompt_wav_upload/prompt_wav_record: Audio files used as reference.
        """
        prompt_speech = prompt_wav_upload if prompt_wav_upload else prompt_wav_record
        prompt_text_clean = None if len(prompt_text) < 2 else prompt_text

        audio_output_path = run_tts(
            text,
            model,
            prompt_text=prompt_text_clean,
            prompt_speech=prompt_speech
        )
        return audio_output_path

    # Define callback function for creating new voices
    def voice_creation(text, gender, pitch, speed):
        """
        Gradio callback to create a synthetic voice with adjustable parameters.
        - text: The input text for synthesis.
        - gender: 'male' or 'female'.
        - pitch/speed: Ranges mapped by LEVELS_MAP_UI.
        """
        pitch_val = LEVELS_MAP_UI[int(pitch)]
        speed_val = LEVELS_MAP_UI[int(speed)]
        audio_output_path = run_tts(
            text,
            model,
            gender=gender,
            pitch=pitch_val,
            speed=speed_val
        )
        return audio_output_path

    with gr.Blocks() as demo:
        # Use HTML for centered title
        gr.HTML('<h1 style="text-align: center;">Spark-TTS by SparkAudio</h1>')
        with gr.Tabs():
            # Voice Clone Tab
            with gr.TabItem("Voice Clone"):
                gr.Markdown(
                    "### Upload reference audio or recording"
                )

                with gr.Row():
                    prompt_wav_upload = gr.Audio(
                        sources="upload",
                        type="filepath",
                        label="Choose the prompt audio file, ensuring the sampling rate is no lower than 16kHz.",
                    )
                    prompt_wav_record = gr.Audio(
                        sources="microphone",
                        type="filepath",
                        label="Record the prompt audio file.",
                    )

                with gr.Row():
                    text_input = gr.Textbox(
                        label="Text", lines=3, placeholder="Enter text here"
                    )
                    prompt_text_input = gr.Textbox(
                        label="Text of prompt speech (Optional; recommended for cloning in the same language.)",
                        lines=3,
                        placeholder="Enter text of the prompt speech.",
                    )

                audio_output = gr.Audio(
                    label="Generated Audio", autoplay=True, streaming=True
                )

                generate_buttom_clone = gr.Button("Generate")

                generate_buttom_clone.click(
                    voice_clone,
                    inputs=[
                        text_input,
                        prompt_text_input,
                        prompt_wav_upload,
                        prompt_wav_record,
                    ],
                    outputs=[audio_output],
                )

            # Voice Creation Tab
            with gr.TabItem("Voice Creation"):
                gr.Markdown(
                    "### Create your own voice based on the following parameters"
                )

                with gr.Row():
                    with gr.Column():
                        gender = gr.Radio(
                            choices=["male", "female"], value="male", label="Gender"
                        )
                        pitch = gr.Slider(
                            minimum=1, maximum=5, step=1, value=3, label="Pitch"
                        )
                        speed = gr.Slider(
                            minimum=1, maximum=5, step=1, value=3, label="Speed"
                        )
                    with gr.Column():
                        text_input_creation = gr.Textbox(
                            label="Input Text",
                            lines=3,
                            placeholder="Enter text here",
                            value="You can generate a customized voice by adjusting parameters such as pitch and speed.",
                        )
                        create_button = gr.Button("Create Voice")

                audio_output = gr.Audio(
                    label="Generated Audio", autoplay=True, streaming=True
                )
                create_button.click(
                    voice_creation,
                    inputs=[text_input_creation, gender, pitch, speed],
                    outputs=[audio_output],
                )

    return demo

# def parse_arguments():
#     """
#     Parse command-line arguments such as model directory and device ID.
#     """
#     parser = argparse.ArgumentParser(description="Spark TTS Gradio server.")
#     parser.add_argument(
#         "--model_dir",
#         type=str,
#         default="pretrained_models/Spark-TTS-0.5B",
#         help="Path to the model directory."
#     )
#     parser.add_argument(
#         "--device",
#         type=int,
#         default=0,
#         help="ID of the GPU device to use (e.g., 0 for cuda:0)."
#     )
#     parser.add_argument(
#         "--server_name",
#         type=str,
#         default="0.0.0.0",
#         help="Server host/IP for Gradio app."
#     )
#     parser.add_argument(
#         "--server_port",
#         type=int,
#         default=7860,
#         help="Server port for Gradio app."
#     )
#     return parser.parse_args()

if __name__ == "__main__":
    # Parse command-line arguments

    # Build the Gradio demo by specifying the model directory and GPU device
    demo = build_ui(
        model_dir="pretrained_models/Spark-TTS-0.5B",
        device=0
    )

    # Launch Gradio with the specified server name and port
    demo.launch(
       share=True,debug=True
    )